# Features for BASIC SEVA plasmids

## Aims and objectives

- [x] Remove features from [BASIC_SEVA genbank files](/sequences/genbank_files/BASIC_SEVA_collection).
- [x] Remove constructs containing p15a origin of replication as contains a "t" insertion.
- [x] Auto-annotate genbank files using benchling.
- [x] Remove color annotations and add description.
- [x] Upload genbank files to JBEI-ICE.
- [ ] Generate a dictionary linking Part ID numbers to SEVA nomenclature.

In [26]:
from Bio import SeqIO
from pathlib import Path
from basicsynbio.main import DEFAULT_ANNOTATIONS
import csv


In [31]:
BSEVA_DIR_INIT = Path.cwd().parent / "sequences" / "genbank_files" / "BASIC_SEVA_collection" / "initial_files"
bseva_seqrecs = []
for gb_file in (BSEVA_DIR_INIT).iterdir():
    # Don't import genbank files with 2nd digit = 6. Avoid processing p15a oris.
    if "BASIC_SEVA_no_annotations.gb" not in gb_file.name and int(gb_file.name[-4]) != 6:
        seqrec = SeqIO.read(gb_file, "genbank")
        seqrec.features = []
        seqrec.annotations = DEFAULT_ANNOTATIONS
        seqrec.id = "BASIC_" + seqrec.id
        seqrec.name = seqrec.id
        bseva_seqrecs.append(seqrec)
SeqIO.write(bseva_seqrecs, BSEVA_DIR_INIT / "BASIC_SEVA_no_annotations.gb", "genbank")

42

## Objective for cell below

Remove color annotations and add description

In [38]:
MARKER_DICT = {
    "1": "Ampicillin",
    "2": "Kanamycin",
    "3": "Chloramphenicol",
    "4": "Streptomycin",
    "5": "Tetracycline",
    "6": "Gentamicin"
}
ORI_DICT = {
    "2": "RK2",
    "3": "pBBR1",
    "4": "pRO1600/ColE1",
    "5": "RSF1010",
    "6": "p15A",
    "7": "pSC101",
    "8": "pUC",
    "9": "pBR322-ROP"
}


def bseva_description(seqrec):
    """Generate string describing BASIC_SEVA collection member."""
    return f"BASIC SEVA vector containing {MARKER_DICT[seqrec.id[-2]]} resistance marker and {ORI_DICT[seqrec.id[-1]]} origin of replication."

seqrecs = []
path_to_gbs = Path.cwd().parent / "basicsynbio" / "parts_linkers"   
for gb_file in path_to_gbs.iterdir():
    if gb_file.is_file and gb_file.name[-4].isdigit():
        seqrec = SeqIO.read(gb_file, "genbank")
        seqrec.description = bseva_description(seqrec)
        for feature in seqrec.features:
            feature.qualifiers = {"label": feature.qualifiers["label"]}
        seqrecs.append(seqrec)
        SeqIO.write(seqrec, gb_file, "genbank")

## Objectives for cell below

Generate a csv file for submission of genbank files to JBEI-ICE.

In [36]:
def jbei_ice_record(seqrec, 
pi="Paul Freemont",
creator = "Matthew C Haines",
creator_email = "mh2210@ic.ac.uk",
status = "In Progress",
biosafety = "Level 1",
host = "E. coli",
circular = "true"
):
    """Accepts a seqrec and metainformation and returns a list of elements corresponding to a record in a JBEI-ICE csv submission file."""
    return [pi, "", "", "", biosafety, seqrec.id, "", "", seqrec.description, "", "", "", status, creator, creator_email, circular, "", "", host, ORI_DICT[seqrec.id[-1]], MARKER_DICT[seqrec.id[-2]], "", seqrec.id + ".gb", ""]


jbei_ice_dir = Path.cwd().parent / "jbei-ice_submissions"
with open(jbei_ice_dir / "jbei-ice_plasmid_csv_upload.csv", "r", newline="") as template_file:
    with open(jbei_ice_dir / "2020-07-13_jbei-ice_submission" / "2020-07-13_jbei-ice_submission.csv", "w", newline="") as submission_file:
        csv_reader = csv.reader(template_file)
        csv_writer = csv.writer(submission_file)
        for row in csv_reader:
            csv_writer.writerow(row)
        for seqrec in seqrecs:
            csv_writer.writerow(jbei_ice_record(seqrec))

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/haine/github_repos/LondonBiofoundry/basicsynbio/jbei-ice_submissions/2020-07-13_jbei-ice_submission/2020-07-13_jbei-ice_submission.csv'

## Objectives for cell below

Generate a dictionary linking Part ID numbers to SEVA nomenclature.

In [34]:

seva_ind = 12
ice_py_loc = Path.cwd().parent / "basicsynbio" / "parts_linkers" / "ice.py"
with open(ice_py_loc, "w") as ice_py_file:
    ice_py_file.write("BSEVA_ICE_DICT = {\n")
    for ind in range(42):
        ice_num = 17297 + ind
        while str(seva_ind)[-1] in [str(num) for num in [0, 1, 6]]:
            seva_ind += 1
        ice_py_file.write(f"    \"{seva_ind}\": \"{ice_num}\",\n")
        seva_ind += 1
    ice_py_file.write("}")
